## ROS 2 project from scratch

This notebook will go through how to create and run a ROS 2 project, covering different fundamentals. This is not *directly* related to the lab work, but rather aimed to give a deeper understanding of how things work in the background. Additionally, going through this notebook on your own computer will also make you more comfortable using the terminal.

### Prerequisites 

**It is assumed that you have already installed ROS 2** If not, download from (here)[https://docs.ros.org/en/jazzy/Installation/Ubuntu-Install-Debs.html] if you have Ubuntu 24.xx or (here)[https://docs.ros.org/en/humble/Installation/Ubuntu-Install-Debs.html] if you use Ubuntu 22.xx.

Before we start we need to make sure the *bashrc* script is set up correctly. Bashrc is a script that is automatically ran each time you open a new terminal window. To get access to ros2 commands in the terminal, we need to *source* ROS. This is a command we will like to have in bashrc. First open a terminal window (ctrl+alt+t). Find your ROS 2 version by
```bash
cd /opt/ros/
ls
```
where you would likely see humble or jazzy. This is your ROS 2 version. Next install *gedit* by running
```bash
sudo apt install gedit
``` 
then
```bash
gedit ~/.bashrc
```
There is a lot going on in this script. Ignore it and scroll to the bottom. The line you will want to paste in at the bottom is
```bash
source /opt/ros/humble/setup.bash # if your version is humble
source /opt/ros/jazzy/setup.bash # if your version is jazzy
```

Another nice command to have is *tree*. Install by running
```bash
sudo apt install tree
```
Tree will allow us to look at the folder structure in a nice way.

### Part 1: Creating the workspace
Doing a project in ROS 2 requires a specific folder structure, which looks something like 
```bash
workspace
    ├── build
    ├── install
    ├── log
    └── src
        └── ros2_packages
```
We start by creating the workspace using the *mkdir* command
```bash
cd ~
mkdir workspace
```
Navigate into the *workspace* folder by using the *cd* command
```bash
cd workspace
```
Now create the src directory (build, install and log will be created automatically later)
```bash
mkdir src
```

### Part 2: Creating a ROS 2 package

From the `src` directory, do
```bash
ros2 pkg create my_package
```
Navigate into the package with 
```bash
cd my_package
```
Now try the tree command by simply doing 
```bash
tree
```
which should result in something like
```bash
.
├── CMakeLists.txt
├── include
│   └── my_package
├── package.xml
└── src
```
*include* and *src* are related to C++ code, so since we are going to use Python they can be safely deleted.
```bash
rm -r include src
```
Now running
```bash
ls
```
should only show CMakeLists.txt and package.xml. CMakeLists.txt can be looked at as a recipe for how to build the package, while package.xml contains information and metadata about the package. The last thing we need to do before we begin writing code is set up the directory the code should be in. In a Python package it is required to have a directory with the **same name as the package** and a file called `__init__.py`. In this case this will look like
```bash
mkdir my_package
cd my_package
# a new file can be created with touch
touch __init__.py
touch my_node.py
```

Go back to your workspace folder and run tree by 
```bash
cd ~/workspace
tree
```
which should look like
```bash
.
└── src
    └── my_package
        ├── CMakeLists.txt
        ├── my_package
        │   ├── __init__.py
        │   └── my_node.py
        └── package.xml
```

### Step 3: Create a ROS 2 node
From the workspace folder, go into vscode with
```bash
code src/
```
Select `my_node.py` as that is the one we will write the node in. Start by adding
```python
#!/usr/bin/env python3
```
at the top of your script. This is required for ROS 2 to run this file. Then we need to add some imports.
```python
import rclpy
from rclpy.node import Node
```
*rclpy* is required to start up the node. Since ROS 2 programming is done in an object-oriented way using classes, we can use the **Node** class as a parent which gives us many nice functionalities right off the bat. Create the class for your node, and make sure to inherit from `Node`.
```python
class MyNode(Node):
    def __init__(self):
        super().__init__("my_node")

        self.get_logger().info("My node is running")
```
If you are not familiar with object-oriented programming in Python, `__init__` is a *method* (method is a function that belongs to a class) that runs when object is created, kind of like setting initial values. `super().__init__()` calls the `__init__` method of the parent class (Node), and requires the node name as input. The last thing to do is initialize and run (called spin in ROS 2) the node.

```python
if __name__ == "__main__": # This just makes sure the code is not executed unless this script is run
    rclpy.init()
    node = MyNode()
    rclpy.spin(node)
    rclpy.shutdown()
```

To check if you successfully created your ROS 2 node, try to run the file. If you got something like this as your output you succeeded.
```bash
[INFO] [1737488305.226680498] [my_node]: My node is running
```
*Kill* the node using ctrl+C in the vscode terminal that popped up at the bottom.

### Part 4: Build the package
Recall that CMakeLists.txt was like a recipe for building. Before we try to build the package, we need to modify it so that it does what we want. Go into the CMakeLists.txt file and delete everything we dont have use for, so that it looks something like this.
```cmake
cmake_minimum_required(VERSION 3.8)
project(my_package)

find_package(ament_cmake REQUIRED)

ament_package()
```

A nice variable to use in CMakeLists.txt is the project name, which can be access with `${PROJECT_NAME}`. Modify the file so it looks something like this:

```cmake
cmake_minimum_required(VERSION 3.8)
project(my_package)

find_package(ament_cmake REQUIRED)

ament_python_install_package(${PROJECT_NAME})

install(PROGRAMS 
  ${PROJECT_NAME}/my_node.py 
  DESTINATION lib/${PROJECT_NAME}
)

ament_package()
```

You are now ready to *build* the package. Head back to the terminal and make sure you are in the workspace directory. Build your package by running 
```bash
colcon build --packages-select my_package
```
After building we need to let the environment know about our newly installed package. We do this by sourcing
```bash
source install/setup.bash
```
Now try running your node
```bash
ros2 run my_package my_node.py
```

### Step 5: Creating a subscriber
The node we just created is very basic and does not do anything. Recall from notebook **00_ros2_project_introduction.ipynb** that different kinds of data is sent between nodes in a *publish-subscriber* manner. The data is sent to a topic, and nodes that want to retrive the data can subscribe to the topic. To build on our new node, let us create a subscriber. Before we do that, imagine you are going to send a package in the mail to your friend who lives on the other side of the country. Before sending the mail you need to decide how to encapsulate the package, for example with a box or just an envelope. Then you have to decide where you are going to send the package; probably to the nearest post office. The same thing applies here. We need to decide which data type the data will be store in, as well as where the data will be sent so that our subscriber can retrive it. Unlike the case where you send a package to your friend, in ROS 2 we have to decide what *happens* when a new mail (message) is received. This logic happens in a *callback*, which is a method that is called when a message is received.

In this example we will use `Float64` message type. Running
```bash
ros2 interface show std_msgs/msg/Float64 --no-comments 
```
shows that this type is built up like this:
```bash
float64 data
```
This means that when we want to send data with this type we populate the `data` slot with a float number. Lets import this type to our code by adding
```python
from std_msgs.msg import Float64
```
to the imports.

It is normal to create the subscriber in the `__init__` method, and a subscriber takes atleast 4 arguments:
* The message type (Float64)
* The topic (/number)
* The callback (self.number_callback)
* A QoS profile (not important)

Adding this means the script looks something like this:

```python
#!/usr/bin/env python3

import rclpy
from rclpy.node import Node
from std_msgs.msg import Float64

class MyNode(Node):
    def __init__(self):
        super().__init__("my_node")

        topic = "/number"

        self.create_subscription(Float64, topic, self.number_callback, 10)

        self.get_logger().info("My node is running, but now it can subscribe")

    def number_callback(self, msg: Float64):
        number = msg.data # Retrieving the number from the data slot in the message
        self.get_logger().info(f"Received {number}")

if __name__ == "__main__":
    rclpy.init()
    node = MyNode()
    rclpy.spin(node)
    rclpy.shutdown()
```

Try to rebuild the package and run it again.
```bash
cd ~/workspace
colcon build --packages-select my_package
source install/setup.bash
ros2 run my_package my_node.py
```
You may notice that nothing is actually happening other than the node running. This is because we are subscribing to a topic, but no one is actually publishing to that topic. To fix that, we can create another node which is responsible for publishing to our topic.

### Step 6: Creating a publisher
Start by copying over the entire script from `my_node.py` and remove some parts so it looks like this:
```python
#!/usr/bin/env python3

import rclpy
from rclpy.node import Node
from std_msgs.msg import Float64

class MyNode(Node):
    def __init__(self):
        super().__init__("my_publisher")

        topic = "/number"

        self.get_logger().info("My node is running")

if __name__ == "__main__":
    rclpy.init()
    node = MyNode()
    rclpy.spin(node)
    rclpy.shutdown()
```
To create the publisher, similarily to the subscriber, we need to give message type, topic and QoS, but not a callback. Additionally we will make it a member of the class since we need to interact with it.
```python
self.number_publisher = self.create_publisher(Float64, topic, 10)
```

To publish a number with the number_publisher, call on its *publish method*.

```python
msg = Float64()
msg.data = 10.0
self.number_publisher.publish(msg)
```
Add both those to the `__init__` method, so it looks something like this:
```python
    def __init__(self):
        super().__init__("my_publisher")

        topic = "/number"

        self.number_publisher = self.create_publisher(Float64, topic, 10)

        msg = Float64()
        msg.data = 10.0
        self.number_publisher.publish(msg)

        self.get_logger().info("My publisher is running")
```

In the CMakeLists.txt, add the new python file.
```cmake
install(PROGRAMS 
  ${PROJECT_NAME}/my_node.py 
  ${PROJECT_NAME}/my_publisher.py 
  DESTINATION lib/${PROJECT_NAME}
)
```

Rebuild the package and run the subscriber node.
```bash
cd ~/workspace
colcon build --packages-select my_package
source install/setup.bash
ros2 run my_package my_node.py
```

In another terminal (ctrl+shift+o in terminator, ctrl+shift+t in terminal) source the workspace and run the publisher.
```bash
source install/setup.bash
ros2 run my_package my_publisher.py
```
If your subscriber node received the number, you have succeeded. However, since `__init__` is only called once, the publisher only publishes one message. To fix this we can add a timer which runs a timer callback every x seconds. This will allow us to publish a message at a chosen frequency. It could look like this:
```python
    timer_period = 0.5 # seconds
    self.create_timer(timer_period, self.timer_callback)
  .
  .
  .
def timer_callback(self):
    msg = Float64()
    msg.data = 10.0
    self.number_publisher.publish(msg)
```
Rebuild, source and run the nodes. Expected output now is a message received every 0.5 seconds :D

**Challenge:** Make the publisher publish *random* numbers (hint: use NumPy) and make the subscriber sum up all the incoming numbers.


### Summary

After going through this notebook, you have
* gotten more comfortable and familiar with the terminal, learning import commands
* learned how to make a package in ROS 2
* learned how to make a node in Python
* made two nodes that can exchange messages

If there are anything that is unclear or you get errors that you cant fix, don't hesitate to send me a message on teams (andeshog).